Google Cloud 가상 프라이빗 클라우드(VPC) 네트워크 피어링을 설정하면 두 개의 VPC 네트워크가 동일한 프로젝트 또는 동일한 조직에 속하는지 여부와 관계없이 이 두 개의 네트워크 간에 비공개 연결을 사용할 수 있습니다.

VPC 네트워크 피어링을 설정하면 Google Cloud에 SaaS(Software as a Service) 생태계를 구축하여 비공개 공간에서 워크로드 간 커뮤니케이션이 가능하도록 조직 내외부의 다양한 VPC 네트워크에서 비공개로 서비스를 제공할 수 있습니다.

VPC 네트워크 피어링은 다음과 같은 경우에 유용합니다.

네트워크 관리 도메인이 여러 개인 조직.
다른 조직과 피어링하고자 하는 조직.
조직 내에 여러 개의 네트워크 관리 도메인이 있는 경우 VPC 네트워크 피어링을 설정하면 VPC 네트워크 전역에 걸쳐 비공개 공간에서 서비스를 제공할 수 있습니다. 다른 조직에 서비스를 제공하는 경우 VPC 네트워크 피어링을 설정하면 이러한 서비스를 비공개 공간에서 해당 조직에 제공할 수 있습니다.

여러 조직에 걸쳐 서비스를 제공할 수 있는 이 기능은 다른 기업에 서비스를 제공하려는 경우에 유용하며, 원래 구조적으로 또는 인수합병의 결과로 여러 개의 상이한 조직 노드가 있는 기업에서 내부용으로 사용하기에도 유용합니다.

VPC 네트워크 피어링은 외부 IP 주소 또는 VPN을 사용하여 네트워크를 연결하는 방식에 비해 다음과 같은 여러 가지 이점이 있습니다.

네트워크 지연 시간: 비공개 네트워킹은 공개 IP 네트워킹에 비해 지연 시간이 더 짧습니다.
네트워크 보안: 서비스 소유자는 공개 인터넷에 서비스를 노출하지 않아도 되므로 관련 위험을 감수할 필요도 없습니다.
네트워크 비용: 피어링된 네트워크는 커뮤니케이션에 내부 IP를 사용하여 Google Cloud 이그레스 대역폭 비용을 절약할 수 있습니다. 모든 트래픽에는 여전히 일반 네트워크 가격 책정이 적용됩니다.

### 두 개의 프로젝트에 커스텀 네트워크 만들기

네트워크는 동일한 조직 노드 내에서 같은 또는 상이한 프로젝트의 다른 VPC 네트워크에서 액세스해야 하는 서비스를 호스팅할 수 있습니다.

또는 하나의 조직에서 서드 파티 서비스가 제공하는 서비스에 액세스해야 하는 경우도 있습니다.

프로젝트 이름은 Google Cloud 전반에서 고유하므로 피어링을 설정할 때 조직을 따로 지정할 필요가 없습니다. Google Cloud는 프로젝트 이름을 기반으로 조직을 인식합니다.

이 실습에서 프로비저닝된 프로젝트는 2개이며 첫 번째 프로젝트는 project-A이고 두 번째 프로젝트는 project-B입니다.

1. 각 프로젝트에서 + 아이콘을 클릭하여 새 Cloud Shell을 시작합니다.

2. project-A의 Cloud Shell에서는 project-A에 해당하는 프로젝트 ID를 설정합니다.

3. project-B의 Cloud Shell에서는 project-B에 해당하는 프로젝트 ID를 설정합니다.
>gcloud config set project [PROJECT-A ID]
>gcloud config set project [PROJECT-B ID]

project-A:
1. 첫 번째 Cloud Shell로 돌아가서 다음을 실행하여 커스텀 네트워크를 만듭니다.
>gcloud compute networks create network-a --subnet-mode custom

2. 다음을 실행하여 이 VPC 내에 서브넷을 만들고 리전과 IP 범위를 지정합니다.
>gcloud compute networks subnets create network-a-subnet --network network-a \
    --range 10.0.0.0/16 --region us-east4

3. VM 인스턴스를 만듭니다.
>gcloud compute instances create vm-a --zone us-east4-c --network network-a --subnet network-a-subnet --machine-type e2-small

4. 연결 테스트 중에 VM과 통신하려면 시큐어 셸이 필요하므로 다음을 실행하여 SSH와 icmp를 사용 설정합니다.
>gcloud compute firewall-rules create network-a-fw --network network-a --allow tcp:22,icmp

##### project-B도 동일하게 진행

### VPC 네트워크 피어링 세션 설정
project-A의 network-a와 project-B의 network-b 사이에 VPC 네트워크 피어링을 설정해야 하는 조직의 경우를 생각해 보겠습니다. VPC 네트워크 피어링을 성공적으로 설정하려면 network-a의 관리자와 network-b의 관리자가 별도로 피어링 연결을 구성해야 합니다.
<img src="./image/networkparing.png">


project-A

네트워킹 섹션으로 이동해 왼쪽 메뉴에서 VPC 네트워크 > VPC 네트워크 피어링을 클릭하여 Cloud 콘솔에서 VPC Network Peering(VPC 네트워크 피어링)으로 이동합니다. 그리고 여기에서 다음을 수행합니다.

1. 연결 만들기를 클릭합니다.
2. 계속을 클릭합니다.
3. 이 연결 측의 이름으로 'peer-ab'를 입력합니다.
4. 내 VPC 네트워크에서 피어링하려는 네트워크(network-a)를 선택합니다.
5. 피어링된 VPC 네트워크 라디오 버튼을 다른 프로젝트 내로 설정합니다.
6. 두 번째 프로젝트의 프로젝트 ID에 붙여넣습니다.
<img src="./image/gcp_paringsetting.png">

##### project-B도 동일하게 진행

VPC 네트워크 피어링이 ACTIVE(활성) 상태로 바뀌고 경로가 교환됩니다. 피어링이 활성 상태로 전환되는 즉시 트래픽 흐름이 다음과 같이 설정됩니다.

피어링된 네트워크의 VM 인스턴스 간: 풀 메시 연결
한 네트워크의 VM 인스턴스에서 피어링된 네트워크의 내부 부하 분산 엔드포인트로
활성 상태의 peer-ab 및 peer-ba 네트워크 다이어그램

이제 피어링된 네트워크 CIDR 프리픽스로 가는 경로를 VPC 네트워크 피어 전반에서 볼 수 있습니다. 이러한 경로는 활성 피어링을 위해 생성되는 묵시적 경로입니다. 이 경로에는 해당되는 경로 리소스가 없습니다. 다음 명령어를 실행하면 project-A의 모든 VPC 네트워크 경로를 확인할 수 있습니다.
>gcloud compute routes list --project [PROJECT ID]

### 연결 테스트

project-A
탐색 메뉴 > Compute Engine > VM 인스턴스를 클릭하여 VM 인스턴스 콘솔로 이동합니다.

vm-a의 [INTERNAL_IP]를 복사합니다.

project-B
탐색 메뉴 > Compute Engine > VM 인스턴스를 클릭합니다.
vm-b 인스턴스에 SSH를 설정합니다.

vm-b의 SSH 셸에서 [INTERNAL_IP_OF_VM_A] 자리에 vm-a 인스턴스 INTERNAL_IP를 입력합니다.

>ping -c 5 [INTERNAL_IP_OF_VM_A]